### 核心想法：
例如对meta-llama/Meta-Llama-3-8B-Instruct进行微调，一般情况下有以下模块可以成为微调模型的选择:
```bash
"target_modules": [
    "gate_proj",
    "down_proj",
    "up_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "q_proj"
  ]
```
当我们进行模型的SFT微调，一般情况选择Lora或者QLora方式，在有以上不同target_modules模块选择的情况下，是否可以对不同的SFT数据集选择不同的target_modules进行微调。</br>
例如：对中文微调数据集选择q_proj，对英文微调数据集选择v_proj，对日语微调数据集选择v_proj...等等此类。

model.merge_and_unload() 的作用是将 LoRA 训练得到的参数 融合 到原始模型的参数中，而不是直接改变模型的结构。

LoRA 训练阶段:
LoRA 并没有直接训练原始模型的参数。相反，它在需要微调的层 (比如 Transformer 中的线性层) 上引入了新的、更小的可训练矩阵 (A 和 B)。
在训练过程中，只有 LoRA 的 A 和 B 矩阵的参数会被更新，而原始模型的参数保持冻结。
由于 A 和 B 矩阵的秩远小于原始权重矩阵，因此 LoRA 可以显著减少需要训练的参数量。
model.merge_and_unload() 操作:

这个操作的核心是将 LoRA 训练得到的 A 和 B 矩阵的低秩分解结果应用到原始模型的权重矩阵上。
具体来说，它会计算 W_new = W_original + B @ A，并将 W_new 作为新的权重矩阵直接更新到原始模型的对应层中。
完成这个操作后，LoRA 的 A 和 B 矩阵会被从内存中卸载，因为它们的信息已经被融合到原始模型的参数中了。
因此，model.merge_and_unload() 虽然改变了原始模型的参数值，但并没有改变模型的结构或层名。 模型的结构定义 (比如哪些层连接在一起) 以及层的名称在整个过程中都保持不变。

你可以把 LoRA 想象成一种 "插件式" 的微调方法：它在训练时像 "插件" 一样附加到原始模型上，但在最终合并后，它的效果会被 "融入" 到原始模型中，而 "插件" 本身会被移除。

In [1]:
import transformers
print(transformers.__version__)

4.40.1


In [ ]:
import logging
import os
from typing import Union, List
import datasets
import torch
from datasets import load_dataset, concatenate_datasets
import transformers


IGNORE_INDEX = -100

logger = logging.getLogger('__name__')

DEFAULT_SYSTEM_PROMPT = """You are a helpful assistant. 你是一个乐于助人的助手。"""
system_format='<|start_header_id|>system<|end_header_id|>\n\n{content}<|eot_id|>'
user_format='<|start_header_id|>user<|end_header_id|>\n\n{content}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'
assistant_format='{content}<|eot_id|>'

def build_instruction_dataset(data_path: Union[List[str],str],
                tokenizer: transformers.PreTrainedTokenizer,
                max_seq_length: int, data_cache_dir = None,
                preprocessing_num_workers = None):

    def tokenization(examples):
        sources = []
        targets = []
        for instruction, input_text, output in zip(examples['instruction'],examples['input'],examples['output']):
            if input_text is not None and input_text !="":
                instruction = instruction+'\n'+input_text
            source = system_format.format(content=DEFAULT_SYSTEM_PROMPT) + user_format.format(content=instruction)
            target = assistant_format.format(content=output)

            sources.append(source)
            targets.append(target)

        tokenized_sources = tokenizer(sources, return_attention_mask=False)
        tokenized_targets = tokenizer(targets, return_attention_mask=False)

        all_input_ids = []
        all_labels = []
        for s,t in zip(tokenized_sources['input_ids'],tokenized_targets['input_ids']):
            input_ids = torch.LongTensor(s + t)[:max_seq_length]
            labels = torch.LongTensor([IGNORE_INDEX] * len(s) + t)[:max_seq_length]
            all_input_ids.append(input_ids)
            all_labels.append(labels)

        results = {'input_ids':all_input_ids, 'labels': all_labels}
        return results


    logging.warning("building dataset...")
    all_datasets = []

    if not isinstance(data_path,(list,tuple)):
        data_path = [data_path]
    for file in data_path:

        if data_cache_dir is None:
            data_cache_dir = str(os.path.dirname(file))
        cache_path = os.path.join(data_cache_dir,os.path.basename(file).split('.')[0]+f"_{max_seq_length}")
        os.makedirs(cache_path, exist_ok=True)
        try:
            processed_dataset = datasets.load_from_disk(cache_path)
            logger.info(f'training datasets-{file} has been loaded from disk')
        except Exception:
            raw_dataset = load_dataset("json", data_files=file, cache_dir=cache_path)
            tokenization_func = tokenization
            tokenized_dataset = raw_dataset.map(
                tokenization_func,
                batched=True,
                num_proc=preprocessing_num_workers,
                remove_columns=["instruction","input","output"],
                keep_in_memory=False,
                desc="preprocessing on dataset",
            )
            processed_dataset = tokenized_dataset
            processed_dataset.save_to_disk(cache_path)
        processed_dataset.set_format('torch')
        all_datasets.append(processed_dataset['train'])
    all_datasets = concatenate_datasets(all_datasets)
    return all_datasets

In [ ]:
import sys
print(sys.executable)

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, PeftModel

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
if torch.cuda.is_available():
    print(f"CUDA is available. Using {torch.cuda.device_count()} GPU(s).")
    print(f"Current device: {torch.cuda.current_device()}")  # 应该输出 0
else:
    print("CUDA is not available.") 

In [ ]:
torch.cuda.set_device(torch.device('cuda:0'))
model_name = "/root/llama3/mode/Meta-Llama-3-8B-Instruct/"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='cuda:0',
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

你提出的问题很好！虽然在加载模型时使用 `torch.bfloat16` 指定了模型参数和计算的精度，但 **通常不需要** 为 tokenizer 也指定 `torch_dtype`。 

原因如下：

1. **Tokenizer 的数据类型:** Tokenizer 主要处理文本数据，例如将文本转换为 token ID，以及将 token ID 转换回文本。这些操作通常使用整数类型（如 `int32`）来表示 token ID，而不是浮点数类型。

2. **Tokenizer 的显存占用:**  Tokenizer 本身的显存占用相对较小，因为它主要存储词汇表和一些预处理规则。相比之下，模型参数和计算过程占用了大部分显存。

3. **框架的默认行为:**  像 Transformers 库中的 `AutoTokenizer` 通常会根据模型的配置自动选择合适的数据类型。 

**总结:**

* 你目前的代码已经很好地将模型加载到了 `bfloat16` 精度，这将有效减少模型的显存占用。
* 不需要为 tokenizer 指定 `torch_dtype`，因为它不会显著影响显存使用。

**其他减少显存占用的方法:**

* **梯度累积 (Gradient Accumulation):**  将多个 mini-batch 的梯度累积起来，再进行一次参数更新，可以模拟更大的 batch size，从而减少显存占用。
* **混合精度训练 (Mixed Precision Training):**  在训练过程中，使用 `float16` 进行计算，同时使用 `float32` 存储模型参数，可以加速训练并减少显存占用。
* **模型量化 (Model Quantization):**  将模型参数和激活值从高精度浮点数转换为低精度整数，可以显著减少模型大小和显存占用。

希望这些信息能够帮助你！ 


In [ ]:
if tokenizer.pad_token is None:
    print("pad_token is None")
    tokenizer.pad_token = tokenizer.eos_token

这段代码的作用是：**如果 tokenizer 没有设置 pad token，则将 pad token 设置为 eos token。**

让我们逐步解释：

1. **`tokenizer.pad_token`**: 
   -  在自然语言处理中，模型通常需要处理长度不一的文本序列。为了方便模型处理，我们会将所有序列填充到相同的长度，而填充的部分就使用 `pad_token` 来表示。
   -  `tokenizer.pad_token` 表示 tokenizer 用来进行填充的特殊 token。

2. **`tokenizer.eos_token`**: 
   -  `eos_token`  代表 "end of sentence"，即句子结束标记符。它通常用于标记一个句子的结束。

3. **`if tokenizer.pad_token is None:`**: 
   -  这行代码首先检查 tokenizer 是否已经设置了 `pad_token`。 

4. **`tokenizer.pad_token = tokenizer.eos_token`**: 
   -  如果 `tokenizer.pad_token` 为 `None`，说明 tokenizer 还没有设置 `pad_token`，那么就将 `tokenizer.eos_token` 赋给 `tokenizer.pad_token`。

**为什么要这样做？**

- 一些预训练模型的 tokenizer 可能没有默认设置 `pad_token`，但通常会有 `eos_token`。
- 在很多情况下，将 `pad_token` 设置为 `eos_token` 是合理的，因为填充的部分通常出现在序列的末尾，语义上类似于句子结束。

**总结:**

这段代码确保了 tokenizer 有一个有效的 `pad_token`，以便在处理变长序列时进行填充操作。


In [ ]:
tuning_targets = {
    "en_dataset": {
        "target_modules": ["q_proj"],
        "lora_config": LoraConfig(
            r=16,
            lora_alpha=32,
            lora_dropout=0.05,
            bias="none",
            task_type=TaskType.CAUSAL_LM,
        )
    },
    "zh_dataset": {
        "target_modules": ["v_proj"],
        "lora_config": LoraConfig(
            r=16,
            lora_alpha=32,
            lora_dropout=0.05,
            bias="none",
            task_type=TaskType.CAUSAL_LM,
        )
    }
}

在提供的配置中，`"task_type": "CAUSAL_LM"`  表示你正在使用 **PEFT (Parameter-Efficient Fine-Tuning)** 库对一个 **因果语言模型 (Causal Language Model)** 进行微调。 

让我们分别解释这两个部分:

* **PEFT (Parameter-Efficient Fine-Tuning):**  PEFT 是一种技术，用于在微调大型语言模型时减少需要更新的参数数量。这使得微调过程更高效，并且可以减少对计算资源的需求。常见的 PEFT 方法包括 LoRA (Low-Rank Adaptation), Prefix Tuning, Adapter Tuning 等。

* **CAUSAL_LM (Causal Language Model):**  因果语言模型是一种语言模型，它学习预测给定上下文中的下一个词。这种模型也被称为自回归语言模型，因为它的预测只依赖于前面的词。 GPT (Generative Pre-trained Transformer) 系列模型就是因果语言模型的典型例子。

**总结:**

`"task_type": "CAUSAL_LM"`  告诉 PEFT 库你正在微调一个因果语言模型，它会根据这个信息选择合适的优化器、损失函数以及其他相关设置。 


In [ ]:
datasets = { 
    "en_dataset": build_instruction_dataset(
                data_path='/root/llama3/code/datasetsSplitTrain/tigerbot-wiki-qa-zh-1k.jsonl',
                tokenizer=tokenizer,
                max_seq_length=1024,
                data_cache_dir='./datacache',
                preprocessing_num_workers=8
    ),
    "zh_dataset": build_instruction_dataset(
                data_path='/root/llama3/code/datasetsSplitTrain/tigerbot-riddle-qa-1k.jsonl',
                tokenizer=tokenizer,
                max_seq_length=1024,
                data_cache_dir='./datacache',
                preprocessing_num_workers=8
    )
}

`max_seq_length` 参数决定了模型能够处理的最长序列长度，它对模型训练和性能有重要影响。选择合适的 `max_seq_length` 需要权衡以下因素：

**1. 数据集特性:**

* **序列长度分布:** 首先要分析你的英文和中文数据集 (`tigerbot-wiki-qa-zh-100.jsonl` 和 `tigerbot-riddle-qa-100.jsonl`) 中序列长度的分布情况。如果大多数序列长度都在 2048 以内，那么设置 `max_seq_length=2048` 是合理的。如果存在大量超过 2048 的序列，则需要考虑增加 `max_seq_length`。
* **任务类型:**  不同的任务对序列长度的要求不同。例如，问答任务通常需要较长的序列长度来容纳问题和答案，而文本分类任务则可能只需要较短的序列长度。

**2. 模型特性:**

* **模型架构:**  不同的模型架构对序列长度的限制不同。例如，基于 Transformer 的模型通常能够处理更长的序列。
* **模型大小:**  更大的模型通常能够处理更长的序列，但也需要更多的计算资源。

**3. 计算资源:**

* **显存限制:**  `max_seq_length` 越大，模型训练和推理所需的显存就越多。如果你的计算资源有限，就需要限制 `max_seq_length`。
* **训练时间:**  `max_seq_length` 越大，模型训练所需的时间就越长。

**建议:**

1. **分析数据集:** 使用工具 (例如 pandas, matplotlib) 分析数据集中的序列长度分布，找到一个能够覆盖大部分样本的 `max_seq_length` 值。
2. **逐步尝试:**  从一个较小的 `max_seq_length` 值开始 (例如 2048)，逐步增加，观察模型性能和训练时间变化，找到一个最佳平衡点。
3. **考虑资源限制:**  根据你的计算资源 (例如 GPU 显存) 限制 `max_seq_length`，避免出现内存不足错误。

**总结:**

`max_seq_length` 的选择需要综合考虑数据集、模型和计算资源等因素。建议进行实验，找到一个既能满足任务需求，又能高效利用资源的最优值。 


In [ ]:
original_model_attn_0_q_proj_weights = model.model.layers[0].self_attn.q_proj.weight
print("原始模型q_proj参数：")
print(original_model_attn_0_q_proj_weights)
original_model_attn_0_v_proj_weights = model.model.layers[0].self_attn.v_proj.weight
print("原始模型v_proj参数：")
print(original_model_attn_0_v_proj_weights)

In [ ]:
model = prepare_model_for_kbit_training(model) 

你观察到的现象是由于 `prepare_model_for_kbit_training(model)` 函数对模型进行了封装和修改，导致参数的属性发生了一些变化。

**原因分析:**

1. **模型封装:**  `prepare_model_for_kbit_training(model)` 函数通常会使用 k-bit 训练库（例如 `bitsandbytes`）提供的类或函数对原始模型进行封装。这意味着你访问到的 `model.model.layers[0].self_attn.q_proj.weight`  实际上已经是封装后的模型的参数，而不是原始模型的参数。

2. **参数类型转换:** 在 k-bit 训练中，为了实现量化，模型参数的类型可能会被转换为 k-bit 训练库所使用的特定类型。例如，`bitsandbytes` 库会将模型参数转换为 `torch.cuda.HalfTensor` 类型（即 FP16）或自定义的量化类型。

3. **属性隐藏:**  封装后的模型可能会隐藏一些原始参数的属性，例如 `dtype` 和 `requires_grad`，以简化用户接口或避免混淆。这是因为 k-bit 训练库通常会自动处理这些属性，用户无需直接操作。

**解释:**

-  `device='cuda:0'` 属性仍然存在，因为 k-bit 训练通常也在 GPU 上进行，所以模型参数仍然需要位于 GPU 设备上。

-  `dtype=torch.bfloat16, requires_grad=True` 属性消失，是因为 k-bit 训练库可能已经将参数类型转换为其他类型，并且自动处理了梯度计算。

**总结:**

`prepare_model_for_kbit_training(model)` 函数对模型进行了封装和修改，导致参数的属性发生了一些变化。这是为了实现 k-bit 训练的量化和优化，用户通常不需要直接操作这些属性。




In [ ]:
prepare_model_for_kbit_training_count = sum(p.numel() for p in model.parameters())
print(f"prepare_model_for_kbit_training_count': {prepare_model_for_kbit_training_count}")

In [ ]:
def print_lora_details(model, layer_idx=0, module_name="v_proj"):
    original_weight = eval(f"model.model.model.layers[{layer_idx}].self_attn.{module_name}.weight")
    lora_A = eval(f"model.model.model.layers[{layer_idx}].self_attn.{module_name}.lora_A.default.weight")
    lora_B = eval(f"model.model.model.layers[{layer_idx}].self_attn.{module_name}.lora_B.default.weight")
    # 计算 W_new = W_original + B @ A
    lora_B_A = torch.matmul(lora_B, lora_A)
    computed_merged_weight = original_weight + lora_B_A

    print(f"Layer {layer_idx} '{module_name}' details:")
    print(f"Original weight:\n{original_weight}")
    print(f"LoRA A:\n{lora_A}")
    print(f"LoRA B:\n{lora_B}")
    print(f"Lora_B_A:\n{lora_B_A}")
    print(f"Computed merged weight (W_original + B @ A):\n{computed_merged_weight}")

In [ ]:
def print_merged_details(model, layer_idx=0, module_name="v_proj"):
    merged_weight = eval(f"model.model.model.layers[{layer_idx}].self_attn.{module_name}.weight")
    print(f"Merged model weight:\n{merged_weight}")

def get_model_parameter(model, layer_id, proj_type):
  param_str = f"model.model.model.layers[{layer_id}].self_attn.{proj_type}.weight"
  param = eval(param_str)
  print(f"Shape of {proj_type} in layer {layer_id}: {param.shape}")

In [ ]:
for dataset_name, tuning_info in tuning_targets.items(): 
    lora_config = tuning_info["lora_config"]
    lora_config.target_modules = tuning_info["target_modules"]

    peft_model = get_peft_model(model, lora_config)

    total_params = sum(p.numel() for p in peft_model.parameters())
    print(f"Total parameters: {total_params}")

    lora_params = 0
    for n, p in peft_model.named_parameters():
        if "lora" in n:
            lora_params += p.numel()
    print(f"LoRA parameters: {lora_params}")

    original_params = total_params - lora_params
    print(f"Original model parameters: {original_params}")
    print("-" * 30)
    
    target_module_str = '_'.join(tuning_info['target_modules'])
    output_dir = f"./results/{dataset_name}_{target_module_str}"
    
    training_args = TrainingArguments(
        output_dir=output_dir,  
        num_train_epochs=1,
        per_device_train_batch_size=1, 
        gradient_accumulation_steps=1, 
        learning_rate=2e-5,
        warmup_ratio=0.05,
        bf16=True,
        logging_steps=100,
        save_steps=500
    )
    
    print(f"Training on {dataset_name} with target modules: {tuning_info['target_modules']}") 
    trainer = Trainer( 
        model=peft_model, 
        args=training_args, 
        train_dataset=datasets[dataset_name], 
        tokenizer=tokenizer 
    ) 
    trainer.train()
    
    peft_model.save_pretrained(output_dir, torch_dtype=torch.bfloat16)
    
    original_param_count = sum(p.numel() for p in peft_model.parameters())
    print(f"original_param_count': {original_param_count}")

    if target_module_str == "q_proj":
        get_model_parameter(peft_model, 0, 'q_proj')
        print("Before merging:")
        print_lora_details(peft_model, layer_idx=0, module_name="q_proj")
    else:
        get_model_parameter(peft_model, 0, 'v_proj')
        print("Before merging:")
        print_lora_details(peft_model, layer_idx=0, module_name="v_proj")
    
    peft_model.merge_and_unload()
    
    if target_module_str == "q_proj":
        get_model_parameter(peft_model, 0, 'q_proj')
        print_merged_details(peft_model, layer_idx=0, module_name="q_proj")
    else:
        get_model_parameter(peft_model, 0, 'v_proj')
        print_merged_details(peft_model, layer_idx=0, module_name="v_proj")

model = PeftModel.from_pretrained(model, output_dir, torch_dtype=torch.bfloat16)
该代码会对原来的模型再次增加一层base_model.model.base_model.model等等

model = PeftModel.from_pretrained(model, output_dir, torch_dtype=torch.bfloat16) 这行代码的作用是从指定的 output_dir 加载一个经过 PEFT (Parameter-Efficient Fine-Tuning) 微调的模型。

让我们逐一解析这段代码：

PeftModel: 这是来自 PEFT 库的类，用于表示经过 PEFT 微调的模型。它通常是对原始模型的封装，并包含了微调过程中添加的额外参数和模块（例如，LoRA 模块）。
.from_pretrained(model, output_dir, torch_dtype=torch.bfloat16): 这是 PeftModel 类的一个静态方法，用于从预训练的模型文件加载模型。
model: 这个参数指定了原始模型，它可以是一个已经实例化的 PyTorch 模型对象，也可以是一个模型标识符（例如，Hugging Face 模型库中的模型名称）。
output_dir: 这个参数指定了存储 PEFT 微调模型的目录路径。该目录应该包含了微调过程中保存的模型参数和其他相关文件。
torch_dtype=torch.bfloat16: 这个参数指定了加载模型参数时使用的数据类型。 torch.bfloat16 是一种 16 位浮点数类型，可以减少内存占用和加速训练，但可能会损失一些精度。
总的来说，这行代码的作用是：

加载原始模型: 如果 model 参数是一个模型标识符，则会先从 Hugging Face 模型库或其他来源下载并加载原始模型。
加载 PEFT 微调参数: 从 output_dir 目录加载 PEFT 微调过程中保存的模型参数，包括 LoRA 模块的参数和其他相关信息。
构建 PEFT 模型: 使用加载的原始模型和 PEFT 参数，构建一个 PeftModel 对象，该对象封装了原始模型和微调后的参数。
使用 PeftModel.from_pretrained 方法可以方便地加载经过 PEFT 微调的模型，而无需重新执行整个微调过程，从而节省时间和计算资源。